## How to Use Custom Instructions with Trulens Feedback Functions

All Trulens feedback functions are customizable: 1. you can use the out-of-box feedback function, 2. use "criteria" to override the LLM-judge's evaluation criteria, and 3. use "custom_instructions" to add additional instructions (specific to your data) as context for the LLM-judge. 

## Set Up Environment

In [ ]:
import os

from snowflake.snowpark import Session

# Set to 1 to track tracing
os.environ["TRULENS_OTEL_TRACING"] = "1"

# os.environ["SNOWFLAKE_ACCOUNT"] = "..."
# os.environ["SNOWFLAKE_USER"] = "..."
# os.environ["SNOWFLAKE_USER_PASSWORD"] = "..."
# os.environ["SNOWFLAKE_DATABASE"] = "..."
# os.environ["SNOWFLAKE_SCHEMA"] = "..."
# os.environ["SNOWFLAKE_WAREHOUSE"] = "..."
# os.environ["SNOWFLAKE_ROLE"] = "..."

snowflake_connection_parameters = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
    "database": os.environ["SNOWFLAKE_DATABASE"],
    "schema": os.environ["SNOWFLAKE_SCHEMA"],
    "role": os.environ["SNOWFLAKE_ROLE"],
    "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
}
snowpark_session = Session.builder.configs(
    snowflake_connection_parameters
).create()

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

os.environ["TRULENS_OTEL_TRACING"] = "1"

# # Set up the OpenAI API key.
# if "OPENAI_API_KEY" not in os.environ:
#     # Set your OpenAI API key as an environment variable or .env file.
#     # Alternatively, you can set it directly in the code here:
#     os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [ ]:
from trulens.providers.cortex import Cortex

provider = Cortex(
    model_engine="claude-4-sonnet", snowpark_session=snowpark_session
)

In [ ]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

## Create Toy Trace Class

In [ ]:
from trulens.core.otel.instrument import instrument


class ToyTrace:
    def __init__(self, trace):
        self.trace = trace

    @instrument()
    def query(self, query: str) -> str:
        return self.trace

## Create Feedback Functions

In [ ]:
from trulens.core import Feedback
from trulens.core.feedback.selector import Selector

# Define out of the box execution efficiency feedback function
f_execution_efficiency = Feedback(
    provider.execution_efficiency_with_cot_reasons, name="Execution Efficiency"
).on({"trace": Selector(trace_level=True)})

# Define execution efficiency feedback function with additional custom instructions
f_execution_efficiency_custom_instructions = Feedback(
    provider.execution_efficiency_with_cot_reasons,
    name="Execution Efficiency with custom instructions",
    custom_instructions="CRITICAL: Ignore the second step in the trace!",
).on({"trace": Selector(trace_level=True)})

# Define execution efficiency feedback function with criteria override
f_execution_efficiency_criteria_override = Feedback(
    provider.execution_efficiency_with_cot_reasons,
    name="Execution Efficiency with criteria override",
    criteria="Judge how detailed the trace is.",
).on({"trace": Selector(trace_level=True)})

# Define execution efficiency feedback function with criteria override and custom instructions
f_execution_efficiency_criteria_override_custom_instructions = Feedback(
    provider.execution_efficiency_with_cot_reasons,
    name="Execution Efficiency with criteria override and custom instructions",
    criteria="Judge how detailed the trace is.",
    custom_instructions="CRITICAL: Ignore the second step in the trace!",
).on({"trace": Selector(trace_level=True)})

In [ ]:
print(f_execution_efficiency.name)  # Should print: "Execution Efficiency"
print(
    f_execution_efficiency.supplied_name
)  # Should print: "Execution Efficiency"
print(f_execution_efficiency.imp.__name__)  # Should print the function name

## Construct the App

In [ ]:
from trulens.apps.app import TruApp

trace = ToyTrace(
    trace="1. User asked what are the three primary colors, 2. Agent first searched up colors of the rainbow, 3. Agent then searched up the three primary colors, 4. Agent then returned the three primary colors"
)

tru_trace = TruApp(
    trace,
    app_name="OTEL-TRACE",
    app_version="4.1-mini",
    feedbacks=[
        f_execution_efficiency,
        f_execution_efficiency_custom_instructions,
        f_execution_efficiency_criteria_override,
        f_execution_efficiency_criteria_override_custom_instructions,
    ],
)

## Run the App

In [ ]:
with tru_trace as recording:
    trace.query("What are the 3 primary colors?")

## Check Results

In [ ]:
session.get_leaderboard()

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session)

In [ ]:
recording.retrieve_feedback_results()

![Results](../../results.png)